In [1]:
import pandas as pd
import os
from pathlib import Path, PureWindowsPath
from pycps import get_asec

In [2]:
os.environ['CENSUS_API_KEY'] = '65e39e1a42641c529d8c890ff8837033a7ba0da2'

# Load FIPS to state/county file in

In [3]:
filename = 'cbsa2fipsxw.csv'
url = Path(PureWindowsPath('C:\\Users\\woodn\\github\\UCSD_MDS\\DSC267R'))
filepath = url / filename
df_geomap = pd.read_csv(filepath,
                        on_bad_lines = 'warn',
                        low_memory = False
                       ).loc[:,['countycountyequivalent'
                                ,'statename'
                                ,'fipsstatecode'
                                ,'fipscountycode'
                               ]
                            ].rename(columns={'statename':'state'
                                              , 'countycountyequivalent':'county'
                                             }
                                    ).dropna().astype({'fipsstatecode':'int64',
                                                       'fipscountycode':'int64'
                                                      }
                                                     )

In [4]:
df_geomap.sample(3)

,county,state,fipsstatecode,fipscountycode
999,Lea County,New Mexico,35,25
721,Benzie County,Michigan,26,19
1260,Clackamas County,Oregon,41,5


In [5]:
df_geomap.loc[:,'county'] = df_geomap.loc[:,'county']\
                                        .str.replace(' County', '', case=False, regex=True)\
                                        .str.replace(' Parish', '', case=False, regex=True)\
                                        .str.replace(' Municipio', '', case=False, regex=True)\
                                        .str.replace(' city', '', case=False, regex=True)

In [6]:
df_geomap.sample(3)

,county,state,fipsstatecode,fipscountycode
1379,Brown,South Dakota,46,13
392,Knox,Illinois,17,95
1402,Campbell,Tennessee,47,13


In [7]:
df_geomap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1882 entries, 1 to 1882
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   county          1882 non-null   object
 1   state           1882 non-null   object
 2   fipsstatecode   1882 non-null   int64 
 3   fipscountycode  1882 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 73.5+ KB


# Load ASEC data

In [10]:
def pull_asec(year):
    if year < 2010:
        cols = ['GESTFIPS', 'GTCO', 'HUFAMINC']
    else:
        cols = ['GESTFIPS', 'GTCO', 'HEFAMINC']
    df_ret = get_asec(year, cols).rename(columns={'gestfips':'fipsstatecode'
                                                  ,'gtco':'fipscountycode'
                                                 }
                                        )
    df_ret.loc[:,'year'] = year
    return df_ret

In [11]:
start = 2005
for a in range(start,2023):
    if a == start:
        df_asec = pull_asec(a)
    else:
        df_asec =  pd.concat([df_asec, pull_asec(a)], ignore_index=True)

Getting CPS ASEC microdata for 2005
Getting CPS ASEC microdata for 2006
Getting CPS ASEC microdata for 2007
Getting CPS ASEC microdata for 2008
Getting CPS ASEC microdata for 2009
Getting CPS ASEC microdata for 2010
Getting CPS ASEC microdata for 2011
Getting CPS ASEC microdata for 2012
Getting CPS ASEC microdata for 2013
Getting CPS ASEC microdata for 2014
Getting CPS ASEC microdata for 2015
Getting CPS ASEC microdata for 2016
Getting CPS ASEC microdata for 2017
Getting CPS ASEC microdata for 2018
Getting CPS ASEC microdata for 2019
Getting CPS ASEC microdata for 2020
Getting CPS ASEC microdata for 2021
Getting CPS ASEC microdata for 2022


In [13]:
df_asec = df_asec.apply(pd.to_numeric)

In [15]:
df_asec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1654348 entries, 0 to 1654347
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   fipsstatecode   1654348 non-null  int64  
 1   fipscountycode  1654348 non-null  int64  
 2   hufaminc        488599 non-null   float64
 3   year            1654348 non-null  int64  
 4   hefaminc        1165749 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 63.1 MB


# Convert to state/county to match other datasets

In [16]:
df_asec.sample(10)

,fipsstatecode,fipscountycode,hufaminc,year,hefaminc
338000,24,17,16.0,2008,NaN
1365210,35,13,NaN,2019,4.0
232998,46,0,4.0,2007,NaN
406815,34,13,9.0,2009,NaN
1158585,28,0,NaN,2017,3.0
110747,36,47,10.0,2006,NaN
69859,12,103,10.0,2005,NaN
443647,45,0,7.0,2009,NaN
300817,44,0,-1.0,2008,NaN
1300435,34,0,NaN,2019,16.0


In [17]:
df_asec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1654348 entries, 0 to 1654347
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   fipsstatecode   1654348 non-null  int64  
 1   fipscountycode  1654348 non-null  int64  
 2   hufaminc        488599 non-null   float64
 3   year            1654348 non-null  int64  
 4   hefaminc        1165749 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 63.1 MB


In [18]:
mask_new = df_asec['hefaminc'].isna()
df_asec.loc[mask_new, 'hefaminc'] = df_asec.loc[mask_new, 'hufaminc']

In [19]:
df_new = pd.merge(df_asec, 
                  df_geomap, 
                  how = 'left',
                  left_on = ['fipscountycode','fipsstatecode'],
                  right_on = ['fipscountycode','fipsstatecode']
                 )

In [20]:
drop_list = ['hufaminc', 'fipsstatecode', 'fipscountycode']
df_new = df_new.drop(columns = drop_list)

In [21]:
df_new.sample(100)

,year,hefaminc,county,state
1442341,2020,1.0,NaN,NaN
625352,2011,-1.0,NaN,NaN
204036,2007,15.0,NaN,NaN
1064817,2016,10.0,NaN,NaN
86504,2005,9.0,Kent,Delaware
...,...,...,...,...
243771,2007,-2.0,Chesapeake,Virginia
314399,2008,8.0,Cuyahoga,Ohio
1039291,2016,11.0,NaN,NaN
316549,2008,-1.0,NaN,NaN


In [22]:
mask = df_new.loc[:,'hefaminc'] > 0
df_new = df_new.loc[mask,:]

In [23]:
df_new = df_new.dropna()

In [24]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504250 entries, 18 to 1654347
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   year      504250 non-null  int64  
 1   hefaminc  504250 non-null  float64
 2   county    504250 non-null  object 
 3   state     504250 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 19.2+ MB


# Build median for each state, county, year

In [25]:
df_gb = df_new.groupby(by=['state','county','year'], as_index=False)

In [26]:
df_blah = df_gb.median()

In [27]:
df_blah.sample(10)

,state,county,year,hefaminc
1130,Florida,Marion,2012,9.0
2381,Michigan,Genesee,2017,9.0
5181,Wisconsin,Winnebago,2021,13.0
1118,Florida,Lee,2019,11.0
85,Arizona,Cochise,2010,10.0
1166,Florida,Miami-Dade,2022,11.0
4795,Virginia,Chesapeake,2020,15.0
4657,Texas,Potter,2012,8.0
3049,New Jersey,Somerset,2012,15.0
3573,North Carolina,Union,2005,14.0


In [31]:
df_blah = df_blah.astype({'hefaminc':'int64'})

# Write out to dataset

In [32]:
filename2 = 'ASEC_income.csv.gz'
url2 = Path(PureWindowsPath('C:\\Users\\woodn\\github\\datasets'))
filepath2 = url2 / filename2

In [34]:
df_blah.to_csv(filepath2,
               index = False,
               compression = 'gzip'
              )